In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING logs


In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))


In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


In [ ]:
from tensorflow import keras
import tensorflow as tf 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 


In [ ]:
train = keras.utils.image_dataset_from_directory(
    '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)


In [ ]:
validate=keras.utils.image_dataset_from_directory(
    '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    
)

In [ ]:
test=keras.utils.image_dataset_from_directory(
    '/kaggle/input/new-plant-diseases-dataset/test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    
    
    
)

In [ ]:


# def normalize_img(image, label):
#     image = tf.cast(image, tf.float32) / 255.0  # scale to [0,1]
#     return image, label

# train = train.map(normalize_img)
# validate = validate.map(normalize_img)


In [ ]:
for x,y in train:
    print(x,x.shape)
    print(y,y.shape)
    break

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.models import Sequential
model=Sequential()

In [ ]:
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
from tensorflow.keras.layers import Flatten, Dropout, Dense

In [ ]:
model.add(Flatten())
model.add(Dropout(0.25))

In [ ]:
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(38,activation='softmax'))

In [ ]:
from tensorflow import keras

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # ✅ remove .legacy
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model.summary()

In [ ]:
model_history=model.fit(x=train,validation_data=validate,epochs=10)

In [ ]:
train_loos,train_accuracy=model.evaluate(train)
val_loos,val_accuracy=model.evaluate(validate)

In [ ]:
model_history.history

In [ ]:
#model.save(trained_model.h5)
# Save as HDF5 format
# model.save('trained_model.h5')

# Or save in Keras format (TensorFlow SavedModel)
model.save('trained_model.keras')


In [ ]:
import json
with open("training_history.json","w") as f:
    json.dump(model_history.history,f)

In [ ]:
epochs=[i for i in range(1,11)]
plt.plot(epochs,model_history.history['accuracy'],color='red',label='training accuracy')
plt.plot(epochs,model_history.history['val_accuracy'],color='blue',label='validation accuracy ')
plt.xlabel('epoches')
plt.ylabel('accuracy')
plt.title('visualisation of accuracy ')
plt.legend()
plt.show()

In [ ]:
class_name=validate.class_names
class_name

In [ ]:
test_set=keras.utils.image_dataset_from_directory(
     '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    
)

In [ ]:
y_pred=model.predict(test_set) #here y refers to the label assigned to each class by incoding in loding data ;

In [ ]:
predicted_categories =tf.argmax(y_pred,axis=1)
predicted_categories 

In [ ]:
y_true=tf.concat([y for x,y in test_set],axis=0)
print(y_true)
true_category=tf.argmax(y_true,axis=1)
true_category

In [ ]:
from sklearn.metrics import classification_report

# Example usage:
print(classification_report(true_category, predicted_categories, target_names=class_name))
